# Demo notebook for Kamodo Flythrough "ModelFlythrough" function
The ModelFlythough function flies a user-supplied trajectory through the chosen model data. This is the function the other flythrough functions call once a trajectory is acquired.
You may run the notebook as is if you have the sample data file, but you must
change the 'file_dir', 'output_name', and 'plot_output' variables in block 3 to have the correct file path.

In [ ]:
# Import satellite flythrough code
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import kamodo_ccmc.flythrough.model_wrapper as MW
# The testing data file is available at https://ccmc.gsfc.nasa.gov/RoR_WWW/output_files/KAMODO_DEMO/

In [ ]:
help(SF.ModelFlythrough)

In [ ]:
# Choosing input values for ModelFlythrough function call
# ----------------------------  
model = 'TIEGCM'  # Choose which model to view the example for
file_dir = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Data/'  # full file path to where the model output data is stored
variable_list = ['T_n','rho','TEC','W_Joule','HmF2']  # list of desired variable names 
# Not all variables in the list will be available in the file(s) found.
coord_sys = 'GEO-sph'  # GEO spherical coordinates as the sample coordinate system for trajectory
# See https://sscweb.gsfc.nasa.gov/users_guide/Appendix_C.shtml for a description of coordinate types
# Choose from any option available in SpacePy.

# Choose naming convention for output files
output_name = 'C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM.nc' 
plot_coord = 'GSE'  # coordinate system chosen for output plots

In [ ]:
# Option 1 for creating input trajectory coordinate arrays:
# Create sample coordinate input arrays, geostationary-like orbit for simplicity (but much lower altitude)
import numpy as np
sat_time = np.linspace(1506039600,1506124800+86400*7,int(86400*7/30+1), dtype=int)  
# Selected time range available in data @ 30 sec intervals
# The chosen time range should match the length of time in the model data files.
# Time values found not to be contained in the model data are automatically discarded (see output of block 10).

sat_lon = np.repeat(0., int(86400*7/30+1))  #in degrees
sat_lat = np.repeat(0., int(86400*7/30+1))  #in degrees
sat_radius = np.repeat(1.08, int(86400*7/30+1))  #in R_earth (~500 km altitude)
# if coord_grid is chosen to be cartesian ('car'), then lon, lat, radius -> x, y, z in the chosen coordinate system
# coordinate arrays must all be the same length
print(len(sat_time), len(sat_lon), len(sat_lat), len(sat_radius))

# Convert sat_radius to km and print for sanity check
from astropy.constants import R_earth
sat_alt = sat_radius[0]*R_earth.value-R_earth.value  # in meters
print(sat_alt/1000., 'km')  # Rough altitude printed in km

In [ ]:
# Option 2 for creating input trajectory coordinate arrays:
# Pull a real satellite trajectory and just change the time values to match the model data.
help(SF.SatelliteTrajectory)

In [ ]:
# Demonstrate how to use the flythrough function to alter a real satellite trajectory
# ---------------------------------------------------------------------------
# The time series for each coordinate is returned in a dictionary
# Use https://sscweb.gsfc.nasa.gov/ to find the satellite name and time range desired
traj_dict, new_coord_sys = SF.SatelliteTrajectory('grace1', 1426660000.0, 1426880700.0, coord_type='GSE')

# Change the time range by subtracting the appropriate value (the diff between the first in each)
start1, start2 = min(sat_time), min(traj_dict['sat_time'])  # start1 can equal the minimum UTC timestamp from the csv file
if start1>start2: 
    diff = start1-start2
    new_time = traj_dict['sat_time']+diff
if start1<start2: 
    diff = start2-start1
    new_time = traj_dict['sat_time']-diff
# The cartesian spatial coordinate time arrays are x=traj_dict['c1'], y=traj_dic['c2'], and z=traj_dict['c3']

In [ ]:
# Run ModelFlythrough with user-created trajectory
# results = SF.ModelFlythrough(model, file_dir, variable_list, sat_time, sat_lon, 
#                            sat_lat, sat_radius, coord_sys,
#                           output_name=output_name, plot_coord=plot_coord)

# Run ModelFlythrough with altered grace1 trajectory from SSCWeb
results = SF.ModelFlythrough(model, file_dir, variable_list, new_time, traj_dict['c1'], 
                            traj_dict['c2'], traj_dict['c3'], new_coord_sys,
                           output_name=output_name, plot_coord=plot_coord)
# Open plots in separate internet browser window for interactivity. Nothing will open here.